In [3]:

import os
import sys
import time
import numpy as np
import tensorflow as tf
import cv2
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from utils import label_map_util
from utils import visualization_utils_color as vis_util
from imutils.video import FPS
from imutils.video import WebcamVideoStream


# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = './model/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './proto/label_map.pbtxt'

NUM_CLASSES = 1

# Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def face_detection():

    # Load Tensorflow model
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        sess = tf.Session(graph=detection_graph)

    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    print(detection_boxes)
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')

    # Actual detection.
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Start video stream
    cap = WebcamVideoStream(0).start()
    fps = FPS().start()

    while True:

        frame = cap.read()

        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        expanded_frame = np.expand_dims(frame, axis=0)
        (boxes, scores, classes, num_c) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: expanded_frame})
        # Visualization of the detection
        vis_util.visualize_boxes_and_labels_on_image_array(
            frame,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=2,
            min_score_thresh=0.40)

        cv2.imshow('Detection', frame)
        fps.update()

        if cv2.waitKey(1) == ord('q'):
            fps.stop()
            break
        

    print("Fps: {:.2f}".format(fps.fps()))
    fps.update()
    cap.stop()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    face_detection()


Tensor("detection_boxes:0", dtype=float32)
Fps: 11.98


In [1]:


import os
import sys
import time
import numpy as np
import tensorflow as tf
import cv2
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from utils import label_map_util
from utils import visualization_utils_color as vis_util
from imutils.video import FPS
from imutils.video import WebcamVideoStream

TEMP_TUNER = 1.80
TEMP_TOLERENCE = 70.6

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = './model/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './proto/label_map.pbtxt'

NUM_CLASSES = 1

# Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
def convert_to_temperature(pixel_avg):
    """
    Converts pixel value (mean) to temperature depending upon the camera hardware
    """
    f = pixel_avg / TEMP_TUNER
    c = (f - 32) * 5/9

    
    return f

def process_face(frame):
    frame = frame
    heatmap = frame
    
    heatmap_gray = cv2.cvtColor(heatmap, cv2.COLOR_RGB2GRAY)
    ret, binary_thresh = cv2.threshold(heatmap_gray, 200, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((5, 5), np.uint8)
    image_erosion = cv2.erode(binary_thresh, kernel, iterations=1)
    image_opening = cv2.dilate(image_erosion, kernel, iterations=1)
    

    image_with_rectangles = np.copy(heatmap)
    
    return image_with_rectangles
def face_detection():
    
    # Load Tensorflow model
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        sess = tf.Session(graph=detection_graph)

    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    print(detection_boxes)
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')

    # Actual detection.
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Start video stream
    cap = WebcamVideoStream(0).start()
    fps = FPS().start()
    cap = cv2.VideoCapture(0)
    frame_height = int(cap.get(3))
    frame_width = int(cap.get(4))

    while(cap.isOpened()):
        ret, frame = cap.read()
        frame = cv2.flip(frame, 180)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        expanded_frame = np.expand_dims(frame, axis=0)
        (boxes, scores, classes, num_c) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: expanded_frame})

        width = frame_width
        height = frame_height

        i = 1
        y = int(boxes[0][i][0]*width)
        x = int(boxes[0][i][1]*height)
        h = int(boxes[0][i][2]*height)
        w = int(boxes[0][i][3]*width)
        print(x,y,w,h)
        output = process_face(frame)
        roi = output[x:x+w,y:y+h]
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

            # Mask is boolean type of matrix.
        mask = np.zeros_like(roi_gray)

        # Mean of only those pixels which are in blocks and not the whole rectangle selected
        mean = convert_to_temperature(np.mean(roi_gray))

        # Colors for rectangles and textmin_area
        temperature = round(mean, 2)
        color = (0, 255, 0) if temperature < TEMP_TOLERENCE else (
            255, 255, 255)
        

        # Draw rectangles for visualisation
        output = cv2.rectangle(output, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(output, "{} C".format(temperature), (x, y-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
        if temperature > 110:
            print("image captured")
            cv2.imwrite(filename='saved_img.jpg', img=frame)
            '''cap.release()
            img_new = cv2.imread('saved_img.jpg', cv2.IMREAD_GRAYSCALE)
            img_new = cv2.imshow("Captured Image", img_new)
            cv2.waitKey(1650)
            cv2.destroyAllWindows()
            print("Processing image...")
            img_ = cv2.imread('saved_img.jpg', cv2.IMREAD_ANYCOLOR)
            print("Converting RGB image to grayscale...")
            gray = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
            print("Converted RGB image to grayscale...")
            print("Resizing image to 28x28 scale...")
            print("Resized...")
            img_resized = cv2.imwrite(filename='saved_img-final.jpg', img=gray)'''

        
        cv2.imshow('Thermal', output)
            # out.write(output)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    cap.release()
    # out.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    face_detection()


Instructions for updating:
non-resource variables are not supported in the long term


2022-04-12 14:27:46.244503: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensor("detection_boxes:0", dtype=float32)
836 371 566 999


error: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [37]:

import os
import sys
import time
import numpy as np
import tensorflow as tf
import cv2
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from utils import label_map_util
from utils import visualization_utils_color as vis_util
from imutils.video import FPS
from imutils.video import WebcamVideoStream


# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = './model/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './proto/label_map.pbtxt'

NUM_CLASSES = 1

# Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def face_detection():

    # Load Tensorflow model
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        sess = tf.Session(graph=detection_graph)

    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')

    # Actual detection.
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    cap = cv2.VideoCapture(0)
    frame_height = int(cap.get(3))
    frame_width = int(cap.get(4))
    # Start video stream
    cap = WebcamVideoStream(0).start()
    fps = FPS().start()

    while True:

        frame = cap.read()
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        expanded_frame = np.expand_dims(frame, axis=0)
        (boxes, scores, classes, num_c) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: expanded_frame})
            
        # Visualization of the detection
        vis_util.visualize_boxes_and_labels_on_image_array(
            frame,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=2,
            min_score_thresh=0.40)

        cv2.imshow('Detection', frame)
        fps.update()

        if cv2.waitKey(1) == ord('q'):
            fps.stop()
            break
        

    print("Fps: {:.2f}".format(fps.fps()))
    fps.update()
    cap.stop()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    face_detection()


Fps: 7.62


In [28]:
2.02325955e-02

0.0202325955